In [4]:
import pymysql
import requests,json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 


"""
API functions
"""

# Functions for getting SensorTower data. 
# Store_Intelligence
# Rate Limit: 6 requests per second. 
# Requests should be made to the following base URL, followed by the specific path to the API you want to use.
# Latest Updated Date: 2021.03.02
# Author: Chen



# Download / Revenue Estimates for applications (If run error please try https://api.sensortower-china.com:443)

# Input: 
# os: Operating System (use "ios", "android",or "unified")
# country: Checking CountryCodes_CategoryIDs to select country (use"WW" for worldwide, "US"  for United States)  
# date_granularity: use "daily", "weekly","monthly", or"quarterly",defaults to "daily"
# start_date:(yyyy-mm-dd) 
# end_date: (yyyy-mm-dd)  
# app_ids: (defaults to None) 
# publisher_ids: (defaults to None) 
# Do not give both app_ids and publisher_ids at same time. 

# Output: A dataframe which containz: 
 
# {
#   "sales_report_estimates_key": {
#     "ios": {
#       "aid": "App ID",
#       "cc": "Country Code",
#       "d": "Date",
#       "iu": "iPhone Downloads",
#       "ir": "iPhone Revenue",
#       "au": "iPad Downloads",
#       "ar": "iPad Revenue"
#     },
#     "android": {
#       "aid": "App ID",
#       "c": "Country Code",
#       "d": "Date",
#       "u": "Android Downloads",
#       "r": "Android Revenue"
#     },
#     "unified": {
#       "app_id": "App ID",
#       "country": "Country Code",
#       "date": "Date",
#       "android_units": "Android Downloads",
#       "android_revenue": "Android Revenue",
#       "ipad_units": "iPad Downloads",
#       "ipad_revenue": "iPad Revenue",
#       "iphone_units": "iPhone Downloads",
#       "iphone_revenue": "iPhone Revenue"
#     }
#   }
# }
# If used publisher_ids, all applications made by this publisher will return. 

# Example: 
# get_data_apps = requests_Download_RevenueEstimates('ios', 'US', 'daily', '2020-11-01', '2020-11-04', '1531465966')
# get_data_publishers = requests_Download_RevenueEstimates('ios', 'US', 'daily', '2020-11-01', '2020-11-04', publisher_ids = '1191495496') 

def requests_Download_RevenueEstimates(os, country, date_granularity, start_date, end_date, app_ids = None, publisher_ids = None):
    if app_ids == None and publisher_ids == None:
        return(print('Error, at least one app ID, or one publisher ID is required.'))
        
    elif publisher_ids == None:
        aux_DRE = requests.get('https://api.sensortower-china.com/v1/'
                               +os+'/sales_report_estimates?app_ids='+app_ids+
                               '&countries='+country+
                               '&date_granularity='+date_granularity+
                               '&start_date='+
                               start_date+'&end_date='+
                               end_date+'&auth_token=r7cTTJq5nBSqxMU_WmJo'
                               )
    else:
        aux_DRE = requests.get('https://api.sensortower-china.com/v1/'
                               +os+'/sales_report_estimates?publisher_ids='+publisher_ids+
                               '&countries='+country+
                               '&date_granularity='+date_granularity+
                               '&start_date='+
                               start_date+'&end_date='+
                               end_date+'&auth_token=r7cTTJq5nBSqxMU_WmJo'
                               )
    df_DRE = pd.DataFrame(aux_DRE.json())
    print(aux_DRE.status_code)
    return(df_DRE)


# Top Apps(If run error please try https://api.sensortower-china.com:443) for one day, week, month, quarter. 给出指定日期后，会寻找该日期所在的日，周，月，季度，
# Input:
# os: Operating System (use "ios", "android",or "unified")
# comparision_attribute: use "absolute", "delta", or "transformed_delta"
# time_range: use "day","week", "month", or "quarter"
# measure: use "units" or "revenue"
# device_type: use "iphone", "ipad", or "total" for `ios`, leave blank for `android`, use "total" for `unified`
# category: Checking CountryCodes_CategoryIDs to select category.(use 6014 for game applications)
# date: Date.YYYY-MM-DD format.Auto-changes to the beginning of time_range.Ex: Mondays for weeks, 1st of the month, 1st day of the quarter, 1st day of the year.
# country: Checking CountryCodes_CategoryIDs to select country (use"WW" for worldwide, "US"  for United States)
# limit: Limit how many apps per call. (Default: 25, Max: 2000)

# Output: A dataframe which contains informations of top Apps. 
# The variable custom_tags contains informations about each application. 

# Example_ios_uni: 
# https://api.sensortower-china.com/v1/ios/sales_report_estimates_comparison_attributes?comparison_attribute=absolute&time_range=week&measure=units&device_type=total&category=6014&date=2021-02-08&country=US&limit=25&auth_token=r7cTTJq5nBSqxMU_WmJo
# get_week_AppRanking1 = requests_ios_uni_TopApps_periodic('ios', 'absolute', 'week','units', 'total', '6014', '2021-02-08','US','25') # 2021-02-08---2021-02-14
# get_week_AppRanking2 = requests_ios_uni_TopApps_periodic('ios', 'absolute', 'week','units', 'total', '6014', '2021-02-07','US','25') # 2021-02-01---2021-02-7
# get_week_AppRanking3 = requests_ios_uni_TopApps_periodic('ios', 'absolute', 'week','units', 'total', '6014', '2021-02-03','US','25') # 2021-02-01---2021-02-7


# Example_Android: 
# https://api.sensortower-china.com/v1/android/sales_report_estimates_comparison_attributes?comparison_attribute=absolute&time_range=week&measure=units&category=game&date=2021-02-08&country=US&limit=25&auth_token=r7cTTJq5nBSqxMU_WmJo
# get_week_ranking4 = requests_Android_TopApps_periodic('android', 'absolute', 'week','units', 'game', '2021-02-08','US','25')

def requests_ios_uni_TopApps_periodic(os,comparision_attribute, time_range, measure, device_type, category, date, country, limit):  
    aux_TopApps = requests.get('https://api.sensortower-china.com:443/v1/'
                               +os+'/sales_report_estimates_comparison_attributes?comparison_attribute='+comparision_attribute+
                               '&time_range='+time_range+
                               '&measure='+measure+
                               '&device_type='+device_type+
                               '&category='+category+
                               '&date='+date+
                               '&country='+country+
                               '&limit='+limit+
                               '&auth_token=r7cTTJq5nBSqxMU_WmJo'
                               )
    df_TopApps = pd.DataFrame(aux_TopApps.json())
    print(aux_TopApps.status_code) # Get response messages
    return(df_TopApps)


# def requests_TopApps_customized(os,comparision_attribute, measure, device_type, category, date, end_date, country, limit):  
#     aux_TopApps = requests.get('https://api.sensortower-china.com:443/v1/'
#                                +os+'/sales_report_estimates_comparison_attributes?comparison_attribute='+comparision_attribute+
#                                '&measure='+measure+
#                                '&device_type='+device_type+
#                                '&category='+category+
#                                '&date='+date+
#                                '&end_date='+end_date+
#                                '&country='+country+
#                                '&limit='+limit+
#                                '&auth_token=r7cTTJq5nBSqxMU_WmJo'
#                                )
#     df_TopApps = pd.DataFrame(aux_TopApps.json())
#     print(aux_TopApps.status_code) # Get response messages
#     return(df_TopApps)    


def requests_Android_TopApps_periodic(os,comparision_attribute, time_range, measure, category, date, country, limit):  
    aux_TopApps = requests.get('https://api.sensortower-china.com/v1/'
                               +os+'/sales_report_estimates_comparison_attributes?comparison_attribute='+comparision_attribute+
                               '&time_range='+time_range+
                               '&measure='+measure+
                               '&category='+category+
                               '&date='+date+
                               '&country='+country+
                               '&limit='+limit+
                               '&auth_token=r7cTTJq5nBSqxMU_WmJo'
                               )
    df_TopApps = pd.DataFrame(aux_TopApps.json())
    print(aux_TopApps.status_code) # Get response messages
    return(df_TopApps)


# Top Publishers(If run error please try https://api.sensortower-china.com:443) for one day, week, month, quarter. 给出指定日期后，会寻找该日期所在的日，周，月，季度，
# Input:
# os: Operating System (use "ios", "android",or "unified")
# comparision_attribute: use "absolute", "delta", or "transformed_delta"
# time_range: use "day","week", "month", or "quarter"
# measure: use "units" or "revenue"
# device_type: use "iphone", "ipad", or "total" for `ios`, leave blank for `android` and `unified`
# category: Checking CountryCodes_CategoryIDs to select category.(use 6014 for game applications)
# date: Date.YYYY-MM-DD format.Auto-changes to the beginning of time_range.Ex: Mondays for weeks, 1st of the month, 1st day of the quarter, 1st day of the year.
# country: Checking CountryCodes_CategoryIDs to select country (use"WW" for worldwide, "US"  for United States)
# limit: Limit how many apps per call. (Default: 25, Max: 2000)

# Output: A dataframe which contains informations of top Publishers. 
# The variable apps contains all informations of applications made by each publisher. 

# Example_ios: 
# https://api.sensortower-china.com/v1/ios/top_and_trending/publishers?comparison_attribute=absolute&time_range=week&measure=units&device_type=total&category=6014&date=2021-02-08&country=US&limit=25&auth_token=r7cTTJq5nBSqxMU_WmJo
# get_week_PubRanking1_ios = requests_ios_TopPublishers_periodic('ios', 'absolute', 'week','units', 'total', '6014', '2021-02-08','US','25')

# Example_Android_uni: 
# https://api.sensortower-china.com/v1/android/top_and_trending/publishers?comparison_attribute=absolute&time_range=week&measure=units&category=6014&date=2021-02-08&country=US&limit=25&auth_token=r7cTTJq5nBSqxMU_WmJo
# get_week_PubRanking1_android = requests_Android_uni_TopPublishers_periodic('android', 'absolute', 'week','units', 'game', '2021-02-08','US','25')

def requests_ios_TopPublishers_periodic(os,comparision_attribute, time_range, measure, device_type, category, date, country, limit):  
    aux_TopPublishers = requests.get('https://api.sensortower-china.com/v1/'
                               +os+'/top_and_trending/publishers?comparison_attribute='+comparision_attribute+
                               '&time_range='+time_range+
                               '&measure='+measure+
                               '&device_type='+device_type+
                               '&category='+category+
                               '&date='+date+
                               '&country='+country+
                               '&limit='+limit+
                               '&auth_token=r7cTTJq5nBSqxMU_WmJo'
                               )
    df_TopPublishers = pd.DataFrame(aux_TopPublishers.json())
    print(aux_TopPublishers.status_code) # Get response messages
    return(df_TopPublishers)


def requests_Android_uni_TopPublishers_periodic(os,comparision_attribute, time_range, measure, category, date, country, limit):  
    aux_TopPublishers = requests.get('https://api.sensortower-china.com/v1/'
                               +os+'/top_and_trending/publishers?comparison_attribute='+comparision_attribute+
                               '&time_range='+time_range+
                               '&measure='+measure+
                               '&category='+category+
                               '&date='+date+
                               '&country='+country+
                               '&limit='+limit+
                               '&auth_token=r7cTTJq5nBSqxMU_WmJo'
                               )
    df_TopPublishers = pd.DataFrame(aux_TopPublishers.json())
    print(aux_TopPublishers.status_code) # Get response messages
    return(df_TopPublishers)



# Store Summary: Information for all ios or android.
# Input:
# Output: 


def requests_Store_Summary(os, category, country, granularity, start_date, end_date):  
    aux_SS = requests.get('https://api.sensortower-china.com/v1/'
                               +os+'/store_summary?categories='+category+
                               '&countries='+country+
                               '&date_granularity='+granularity+
                               '&start_date='+start_date+
                               '&end_date='+end_date+
                               '&auth_token=r7cTTJq5nBSqxMU_WmJo'
                               )
    df_SS = pd.DataFrame(aux_SS.json())
    print(aux_SS.status_code) # Get response messages
    return(df_SS)

# Game Breakdown: Information for all ios or android.(games specific)
# Input:
# Output: 

def requests_Games_Breakdown(os, category, country, granularity, start_date, end_date):  
    aux_GB = requests.get('https://api.sensortower-china.com/v1/'
                               +os+'/games_breakdown?categories='+category+
                               '&countries='+country+
                               '&date_granularity='+granularity+
                               '&start_date='+start_date+
                               '&end_date='+end_date+
                               '&auth_token=r7cTTJq5nBSqxMU_WmJo'
                               )
    df_GB = pd.DataFrame(aux_GB.json())
    print(aux_GB.status_code) # Get response messages
    return(df_GB)

# Reviews 
# Input: 
# os: ios or android
# app_id
# start_date
# end_date
# country: US or leave blank

# Output: 
# average: Rating
# total: Total number of person who rates.

# Example: 
# https://api.sensortower-china.com/v1/ios/review/get_ratings?app_id=284882215&start_date=2021-02-08&end_date=2021-02-14&country=US&auth_token=r7cTTJq5nBSqxMU_WmJo
# test_rating1 = requests_Rating('ios', '1540979366', '2020-01-01', '2021-03-16')
# test_rating2 = requests_Rating('android', 'com.sushirolls.app', '2020-01-01', '2021-03-16')

def requests_ios_Rating(os, app_id, start_date, end_date): 
        aux_rating = requests.get('https://api.sensortower-china.com/v1/'
                                   +os+'/review/get_ratings?app_id='+app_id+
                                   '&start_date='+start_date+
                                   '&end_date='+end_date+
                                   '&country=US&auth_token=r7cTTJq5nBSqxMU_WmJo'
                                   )
        df_rating = pd.DataFrame(aux_rating.json())
        print(aux_rating.status_code) # Get response messages
        return(df_rating)
        
def requests_android_Rating(os, app_id, start_date, end_date): 
            aux_rating = requests.get('https://api.sensortower-china.com/v1/'
                                       +os+'/review/get_ratings?app_id='+app_id+
                                       '&start_date='+start_date+
                                       '&end_date='+end_date+
                                       '&auth_token=r7cTTJq5nBSqxMU_WmJo'
                                       )
            df_rating = pd.DataFrame(aux_rating.json())
            print(aux_rating.status_code) # Get response messages
            return(df_rating)        
        
def requests_Rating(os):
    if os == 'ios':
        def aux_func(app_id, start_date, end_date):
            return requests_ios_Rating(os, app_id, start_date, end_date)
      
    else:
        def aux_func(app_id, start_date, end_date): 
            return requests_android_Rating(os, app_id, start_date, end_date)
        
    return aux_func    


"""
除Rating字段外的更新
"""

# 查询并插入新的游戏

def premium_apps_insert(df_ios, df_android, start_date, end_date):
    country = ['US', 'JP', 'TW', 'KR', 'GB', 'CA', 'AU', 'SG', 'HK', 'MO', 'DE', 'WW']
    li_ios_paoku = list(df_ios_paoku[1])
    li_android_paoku = list(df_android_paoku[1])
    a, b, c = '', '', pd.DataFrame()
    
    for i in li_ios_paoku: 
        a = a + i +','
    a = [a[:-1]]
        
        
    for i in li_android_paoku: 
        b = b + i +','
    b = [b[:-1]]
        
    for i in country:
        c = pd.concat([c, requests_Download_RevenueEstimates('ios', i, 'weekly', start_date, end_date, a[0])])
        c = pd.concat([c, requests_Download_RevenueEstimates('android', i, 'weekly', start_date, end_date, b[0])])
    c = c.reset_index(drop=True) # 需要赋值
    c['date'] = c['d'].apply(lambda x: x[0:10]) 
    c['downloads'] = c.apply(lambda x: x['au'] + x['iu'] if str(x['u']) == 'nan' else x['u'], axis = 1)
    c['downloads'] = c['downloads'].where(c['downloads'].notnull(), 0) # change nan to 0, mask()
    c['platform'] = c.apply(lambda x: 'ios' if type(x['aid']) == int else 'android', axis = 1)
    c['country'] = c.apply(lambda x: x['cc'] if type(x['aid']) == int else  x['c'], axis = 1)
    aux_table = pd.DataFrame()
    aux_table['product_id'] = c['aid'].apply(lambda x: str(x))
    aux_table['date'] = c['date']
    aux_table['downloads'] = c['downloads'].apply(lambda x: str(x))
    aux_table['platform'] = c['platform']
    aux_table['country'] = c['country']
    aux_table = aux_table.values.tolist()
    return aux_table

# 更新原始已有游戏的publisher_id
# Sensor Tower
def rename_table(df_stpre):
    df_stpre['product_id'] = df_stpre[1]
    df_stpre['date'] = df_stpre[2]
    df_stpre['publisher_id'] = df_stpre[3]
    df_stpre['platform'] = df_stpre[4]
    df_stpre['country'] = df_stpre[5]
    df_stpre['downloads'] = df_stpre[6]
    df_stpre['rating'] = df_stpre[7]
    df_stpre = df_stpre.drop([1,2,3,4,5,6,7], axis =1)
    return df_stpre

def rename_st(df_st):
    df_st['product_id'] = df_st[1]
    df_st['publisher_id'] = df_st[12]
    df_st = df_st.drop([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19], axis = 1)
    return df_st

def premium_apps_update(df_st, df_stpre):
    
    df_st = rename_st(df_st)
    df_stpre = rename_table(df_stpre)
    result = pd.merge(df_stpre, df_st, how='left', on=['product_id'])
    aux_table = result['publisher_id_y'].values.tolist()
    aux_table = [[k] for k in aux_table] # enumerate
    for k in range(len(aux_table)): 
        aux_table[k] = [aux_table[k]]
    return aux_table    
    

"""
Rating更新
"""

# 更新评分：Updating Rating

def get_Rating(aux_id, os, start_date, end_date):
    table = pd.DataFrame()
    for i in aux_id:
        table = pd.concat([table, requests_Rating(os)(i, start_date, end_date)])
    return table    


def Rating(df_stpre, start_date, end_date):
    df_stpre = rename_table(df_stpre)
    df_aux = df_stpre.drop_duplicates(subset=['product_id'], keep= 'first')
    df_aux_ios = df_aux[df_aux['platform'] == 'ios']
    df_aux_an = df_aux[df_aux['platform'] == 'android']
    aux_ios_id = df_aux_ios['product_id'].values.tolist()
    aux_an_id = df_aux_an['product_id'].values.tolist()
    io = get_Rating(aux_ios_id, 'ios', start_date, end_date)
    io = io.drop_duplicates(subset=['app_id'], keep= 'first')
    an = get_Rating(aux_an_id, 'android', start_date, end_date)
    an = an.drop_duplicates(subset=['app_id'], keep= 'first')
    aux_con = pd.concat([io, an])
    aux_con = aux_con.reset_index(drop = True)
    final = pd.DataFrame()
    final['rating'] = aux_con['average'].apply(lambda x: str(x))
    final['update_date'] = aux_con['date'].apply(lambda x: x[0:10])
    final['product_id'] = aux_con['app_id'].apply(lambda x: str(x))
    final = final.values.tolist()
    return final



In [ ]:
pd.set_option('display.max_rows', 1900)

## st_premium_apps (Downloads) update

In [160]:
# 寻找目标新游戏，插入数据库中

conn = pymysql.connect(host='124.156.172.225', user='powerbi_user', passwd='1powerBI987', db='market_bi')
cur=conn.cursor()
cur.execute('''
            SELECT * FROM st_app_info WHERE platform = 'ios' and subclass LIKE '%跑酷%' 
            '''
           )
test = cur.fetchall()
df_ios = pd.DataFrame(list(test))

cur.execute('''
            SELECT * FROM st_app_info WHERE platform = 'android' and subclass LIKE '%跑酷%' 
            '''
           )
test = cur.fetchall()
df_android = pd.DataFrame(list(test))

In [183]:
aux_table = premium_apps_insert(df_ios, df_android, '2021-1-4', '2021-3-8')


200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


In [199]:
# insert: 往表中插入新游戏时使用
conn = pymysql.connect(host='124.156.172.225', user='powerbi_user', passwd='1powerBI987', db='market_bi')
cur=conn.cursor()
cur.executemany('''
            insert into st_premium_apps(product_id, date, downloads, platform, country) values(%s, %s, %s, %s, %s)
            on duplicate KEY update 
            date = values(date),
            downloads = values(downloads),
            platform = values(platform),
            country = values(country)
            ;
            '''
           , aux_table) # 多行插入使用 executemany()

conn.commit() # for dataframe from python to database.  



In [296]:
# 寻找已有游戏，为更新publisher_id准备

conn = pymysql.connect(host='124.156.172.225', user='powerbi_user', passwd='1powerBI987', db='market_bi')
cur=conn.cursor()
cur.execute('''
            SELECT * FROM st_app_info
            '''
           )
test = cur.fetchall()
df_st = pd.DataFrame(list(test))


cur.execute('''
            SELECT * FROM premium_apps
            '''
           )
test = cur.fetchall()
df_stpre = pd.DataFrame(list(test))

In [ ]:
aux_table = premium_apps_update(df_st, df_stpre)

In [306]:
# Update publisher_id
conn = pymysql.connect(host='124.156.172.225', user='powerbi_user', passwd='1powerBI987', db='market_bi')
cur=conn.cursor()

for i in range(len(aux_table)):
    j = i + 1
    cur.execute(
                f'''
                UPDATE premium_apps SET publisher_id = %s WHERE id = {j}
                ;
                '''
               , aux_table[i]) # 多行插入使用 executemany()

    conn.commit() # for dataframe from python to database.  


## st_premium_apps (Rating) update

In [405]:
# 找到需要游戏更新Rating
conn = pymysql.connect(host='124.156.172.225', user='powerbi_user', passwd='1powerBI987', db='market_bi')
cur=conn.cursor()
cur.execute('''
            SELECT * FROM premium_apps
            '''
           )
test = cur.fetchall()
df_stpre = pd.DataFrame(list(test))


In [423]:
Rating_f = Rating(df_stpre, '2021-1-16', '2021-3-18')

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


In [ ]:
# Update 已有数据Rating字段时使用
conn = pymysql.connect(host='124.156.172.225', user='powerbi_user', passwd='1powerBI987', db='market_bi')
cur=conn.cursor()

for i in range(len(Rating_f)):
    cur.execute(
                f'''
                UPDATE premium_apps SET rating = %s, update_date = %s WHERE product_id = %s
                ;
                '''
               , Rating_f[i]) # 多行插入使用 executemany()

    conn.commit() # for dataframe from python to database.  